In [119]:
from PIL import Image
from torchvision import transforms, models
from model.model import Model
import matplotlib.pyplot as plt
import math
import numpy as np
import torch

In [120]:
im = Image.open('images/cracow.tif')
nim = np.array(im)
nim.shape

(4096, 5120, 3)

In [121]:
transforms = transforms.Compose([            
    transforms.Resize((224, 224), interpolation=Image.NEAREST),
    transforms.ToTensor(),
])

In [122]:
model = Model()
model.load_state_dict(torch.load('model/state_dict.pth', map_location=torch.device('cpu')))
model.eval()

Using cache found in /home/witek/.cache/torch/hub/pytorch_vision_v0.10.0
/home/witek/anaconda3/envs/main/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/witek/anaconda3/envs/main/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model(
  (base): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

In [123]:
# ni = 10
# 
# preds = [model(transforms(image).unsqueeze(0)) for image in images[:ni]]
# 
# fig, ax = plt.subplots(1, ni, figsize=(20, 20))
# for i, pred, image in zip(range(ni), [round(pred.item()) for pred in preds], images[:ni]):
#     ax[i].imshow(image)
#     ax[i].set_title(pred)
#     ax[i].axis('off')
# 
# plt.show()

In [124]:
def classify_image(image):
    return round(model(transforms(image).unsqueeze(0)).item())

def make_red(image):
    image = np.array(image)
    image[:, : , 0] = np.full(image[:, : , 0].shape, 255)
    image = Image.fromarray(image)
    return image

In [ ]:
slice_size = 50
new_image = Image.new("RGB", im.size)

slices = []
for i in range(0, im.size[0], slice_size):
    for j in range(0, im.size[1], slice_size):
        slice = im.crop((i, j, i+slice_size, j+slice_size))
        if classify_image(slice):
            slice = make_red(slice)
        new_image.paste(slice, (i, j))
         
new_image.show()
new_image.save('images/cracow-map.tif')